# Stock Flex
### Technical and Fundamental Portfolio Builder

This Jupyter Notebook will outline the exploration, developement and clean-up of the appliation code

### Table of Contents
- Data connectors
- Data query and clean-up
- Analytical methods
    - Technical
    - Fundamental
    - Traditional
    
---

- Visualization
- Dashboard (GUI)
    - GUI
    - Data filtration methods

### Dependencies

In [1]:
# Data analytics
import pandas as pd
import numpy as np
import panel as pn

# Database tools
import sqlite3
from sqlalchemy import create_engine

# Visualization
# pn.extension('plotly')
# import plotly.express as px
# import hvplot.pandas
import matplotlib.pyplot as plt

# System
import os
import time, sys
from pathlib import Path
from dotenv import load_dotenv
import requests
from datetime import date, datetime, timedelta
from IPython.display import clear_output

# Finance
import alpaca_trade_api as tradeapi
import quandl as ql
import finnhub

import warnings
warnings.filterwarnings('ignore')

## Data Connections
- Static Data Connections
- Dynamic Data Connections

### Static Data Connections

#### Stock Ticker Lists

In [2]:
# Get tickers within S&P500 index
sp500_tickers_path = Path('resources/sp500_tickers.csv')
sp500_tickers = pd.read_csv(sp500_tickers_path).sort_values(by="Symbol")

# Get test tickers within S&P500 index
sp500_tickers_test_path = Path('resources/sp500_tickers_test.csv')
sp500_tickers_test = pd.read_csv(sp500_tickers_test_path)

In [3]:
# Get fundamental data from csv
stock_fundamentals_data_path = Path("resources/fundamental_data.csv")
stock_fundamentals_df = pd.read_csv(stock_fundamentals_data_path)

#### Database Builder

In [4]:
def create_db():
    '''This method creates a SQLite database that will store all of ticker daily prices;
    This method is created to resolve the Alpaca API request limits that can greatly slow down large data requests
    '''
    
    print("This will restore the 'stock_prices' database and delete all of stored values, are you sure? y/[n]")
    confirm = input()
    
    if confirm == 'y':
        conn = sqlite3.connect('resources/stock_prices.db')
        
    else:
        print("Databse creation aborted")

### Dynamic Data Connections

#### Stock Price Data

In [5]:
# Alpaca API connector
load_dotenv("../resources/api_keys.env")

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version = "v2"
)

#### Fundamental Stock Data

In [6]:
# FinnHub API connector
load_dotenv("../resources/api_keys.env")

# Set FinnBug API key
finnhub_api_key = os.getenv("FINNHUB_API_KEY")

# Create FinnHub API object
finnhub_client = finnhub.Client(api_key=finnhub_api_key)

# Method to obtain json data from Polygon API
def finnhub_data(ticker):
    
    data = finnhub_client.company_basic_financials(ticker, "")
    data_df = pd.DataFrame(data)
    time.sleep(1)
    
    return data_df

#### Bond Data

In [7]:
# Treasury bonds
def treasury_data():
    return ql.get("USTREASURY/YIELD")

#### Crypto Data

In [8]:
# Crypto connector URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

# Build out the crypto connector here

## Data Parsing

### Stock Data

In [47]:
# Get prices for tickers withing a given index or sector
def stock_prices_from_api(tickers_df, start_date, end_date):
    '''Returns pd.DataFrame with prices for the given tickers
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    start_date : str() - string with date in following format YYYY-MM-DD
    end_date: str() - string with date in following format YYYY-MM-DD 
    
    
    Returns
    -------
    result_df : pd.DataFrame with securities price data
    '''
   
    # Get list of tickers from the tickers_df list or tickers_df DataFrame 
    tickers = tickers_df["Symbol"]
    
    # Parse start and end dates
    start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()
    
    # Connect to Alpaca API and get data
    """Condition handling: 
        a. Alpaca API 422 Client Error if more than 100 tickers are passed - COMPLETE
        b. Alpaca API data max row limit of 1000 - PENDING"""
    
    
    # a. Alpaca API condition handling, sending 100 tickers at a time
    # Declate a pd.DataFrame
    result_df = pd.DataFrame()
    tickers_n = 50
    
    for i in range(0, len(tickers), tickers_n):
        # Slice the ticker list into lists of 50 tickers
        sliced_tickers = tickers[i:i + tickers_n] 
        
        temp_df = api.get_barset(
        sliced_tickers,
        timeframe = "1D",
        start = start_date,
        end = end_date,
        limit = 1000).df

        # Append temporary dataframe to result_df
        result_df = pd.concat([result_df, temp_df], axis = "columns", join = "outer")
        time.sleep(0.1)
        
    return result_df



# Seed stock price database
def seed_stock_prices_database():
    '''Seeds stock_prices database that houses all prices for tickers within an index or sector'''

    # Get data for dates indicated
    data = stock_prices_from_api(sp500_tickers, "2015-01-01", "2018-01-01")
    
    # Transpose dataframe
    data = data.transpose()
    
    # Connect to databse
    conn = sqlite3.connect('resources/stock_prices.db') 
    c = conn.cursor()
    
    # Iterate over dataframe and populate the database
    for ticker, sub_frame in data.groupby(level=0):
        
        ticker = "_" + ticker.replace('.','_')
        
        list_of_tables = c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{ticker}';").fetchall()
        if list_of_tables == []:
            sub_frame.transpose().to_sql(ticker, conn, if_exists='append')
        else:
            sub_frame.transpose().to_sql(ticker, conn, if_exists='append')
    
    conn.commit()
    conn.close()
            
   

# Update stock price database     
def update_stock_prices_database(tickers_df):
    '''Updates stock_prices database that houses all prices for tickers within an index or sector'''
    
    # Get today's date
    today_date = date.today().strftime("%Y-%m-%d")
    
    # Connect to databse
    conn = sqlite3.connect('resources/stock_prices.db') 
    c = conn.cursor()
    
    # Iterate over dataframe and populate the database
    ticker_count = 0
    for index, ticker_data in tickers_df.iterrows():
        
        ticker = "_" + ticker_data["Symbol"].replace('.','_')
        
        # Get list of all tables in databse
        list_of_tables = c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{ticker}';").fetchall()
        
        # If table does not exist, create and seed
        if list_of_tables == []: # If tables does not exist
            # Get data from Alpaca API
            data = stock_prices_from_api(ticker_data, "2015-01-01", "2018-01-01")
            
            # Write stock data to databse
            data.to_sql(ticker, conn, if_exists='append')
            
        else: # If table does exist
            try:
                # Get last date from the table
                last_date = c.execute(f"SELECT time FROM {ticker} ORDER BY time DESC LIMIT 1").fetchall()
                last_date = ''.join(last_date[0]).split(" ")[0]

                # Get next date
                next_date = datetime.strptime(last_date, '%Y-%m-%d') + timedelta(days=1)

                # Change date format for Alpaca
                next_date = next_date.strftime("%Y-%m-%d")
                
                # Get today's date
                today_date = datetime.today().strftime("%Y-%m-%d")

                # Get data from Alpaca API
                data = stock_prices_from_api(ticker_data, last_date, today_date)

                # Write data to database
                data.to_sql(ticker, conn, if_exists='append')
            except:
                pass
            
        ticker_count += 1
        print(f"Processing {ticker} | {ticker_count}/{len(tickers_df.index)}")
    
    conn.commit()
    conn.close()
    
    
    
# Get stock prices from database
def stock_prices_from_db(ticker_df, start_date, end_date):
    '''Returns a pd.DataFrame with stock price data for tickers passed in ticker_df and 
    filtered by dates passed in start_date and end_date
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    start_date : str() - string with date in following format YYYY-MM-DD
    end_date: str() - string with date in following format YYYY-MM-DD 
    
    
    Returns
    -------
    result_df : pd.DataFrame with securities price data
    '''
    # Connect to database
    conn = sqlite3.connect('resources/stock_prices.db') 
    c = conn.cursor()
    
    # Cast dates to pandas datetime
    start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()
    
    # Declare result_df
    result_df = pd.DataFrame()
    
    # Iterate over tickers
    for ticker in list(ticker_df["Symbol"]):
        
        try:
            # Parse ticker name
            _ticker = "_" + ticker.replace('.','_')

            # Query databse
            data = pd.read_sql_query(f"SELECT * FROM {_ticker}", conn)

            # Filter database by dates
            data = data[(data['time'] >= start_date) & (data['time'] <= end_date)].set_index(['time'])

            # Concatenate dataframes
            result_df = pd.concat([result_df, data], axis = "columns", join = "outer")
        
        except:
            print(f"Ticker {ticker} not found in database.")
        
    return result_df



# Get stock fundamental data from FinnHub
def generate_stock_fundamentals(tickers_df):
    '''Returns pd.DataFrame with fundamentals of tickers within tickers_df
    
    ...
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
        
    Returns
    -------
    result_df : pd.DataFrame - securities fundamental data
    '''
    
    result_df = pd.DataFrame()
    
    for ticker in tickers_df['Symbol']:
        fundamental_data = finnhub_data(ticker)
        result_df = pd.concat([result_df, fundamental_data], axis='rows', join="outer")
        
    # Parse the dataframe
    result_df = result_df.reset_index().set_index('symbol')
    result_df = result_df.drop('metricType', 1)
    result_df.columns = ['metric_type', 'metric', 'series']
    
    return result_df



# Refresh stock fundamental data csv
def refresh_fundamentals_csv(tickers_df):
    '''Returns pd.DataFrame with fundamentals of tickers within tickers_df
    and/or calls to generate a refreashed dataset
    
    ...
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    '''
    
    result_df = stock_fundamentals(tickers_df)
    result_df.to_csv("resources/fundamental_data.csv")

In [50]:
# conn = sqlite3.connect('resources/stock_prices.db') 
# c = conn.cursor()
# data = c.execute("SELECT * FROM _AAPL")
# for row in data:
#     print(row)
# conn.commit()
# conn.close()

data = stock_prices_from_db(sp500_tickers, "2020-12-01", "2021-02-01")
data.head()

Ticker BF.B not found in database.
Ticker BRK.B not found in database.


,"('A', 'open')","('A', 'high')","('A', 'low')","('A', 'close')","('A', 'volume')","('AAL', 'open')","('AAL', 'high')","('AAL', 'low')","('AAL', 'close')","('AAL', 'volume')",...,"('ZION', 'open')","('ZION', 'high')","('ZION', 'low')","('ZION', 'close')","('ZION', 'volume')","('ZTS', 'open')","('ZTS', 'high')","('ZTS', 'low')","('ZTS', 'close')","('ZTS', 'volume')"
time,,,,,,,,,,,,,,,,,,,,,
2020-12-02 00:00:00-05:00,115.63,115.9500,114.13,114.53,1459697,14.16,14.86,13.85,14.850,71953223,...,40.00,41.650,39.96,41.60,2848965,161.00,161.835,158.48,158.96,1350196
2020-12-03 00:00:00-05:00,114.15,115.4229,113.16,113.58,1245876,15.19,16.50,15.16,16.090,166646308,...,41.77,42.750,41.51,42.44,2231270,158.58,159.085,156.44,157.01,1847935
2020-12-04 00:00:00-05:00,113.60,115.2400,113.45,115.17,895806,16.40,16.93,16.12,16.400,108904784,...,44.15,44.300,43.00,44.03,2619855,157.16,158.220,156.66,157.94,1320390
2020-12-07 00:00:00-05:00,115.20,116.4400,114.12,114.40,1531216,16.54,18.07,16.47,17.210,154232320,...,43.94,43.940,42.83,42.91,1517619,157.56,159.200,157.56,158.48,1110001
2020-12-08 00:00:00-05:00,114.34,117.3400,114.33,116.98,968890,17.47,17.92,17.16,17.635,92186969,...,41.90,43.185,41.81,42.97,1352316,156.87,159.500,156.47,158.99,1358891


## Computational Methods

### Technical

In [ ]:
# please start with RSI and MACD
# your method must take entire dataframe and add a RSI or MACD column to it
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)


def rsi(df, days):
    '''Returns a pd.DataFrame with RSI column appended
    
    ...
    
    Parameters
    ----------
    df : pd.DataFrame - dataframe to be processed
    days : int() - numbers of days for RSI calcualtion
    
    Returns
    -------
    result_df : pd.DataFrame - dataframe with RSI column appended, calcualted daily for 
        timeperiod specified by days
    '''
    
    result_df = pd.DataFrame()
    
    # Your code
    
    return result_df

def macd(df, short_ema, long_ema):
    '''Description...
    
    ...
    
    Parameters
    ----------
    df : pd.DataFrame - dataframe to be processed
    short_ema : int() - short-term EMA for MACD calculation
    long_ema : int() - long-term EMA for MACD calculation
    
    Returns
    -------
    result_df : pd.DataFrame - dataframe with MACD column appended, calcualted daily for 
        timeperiod specified by days
    '''
    pass

# Method test

### Fundamental

In [54]:
# Fundamental data filter
def fundamental_data_query(tickers_df, fundamental_indicator_key, lower_bound = -1000000, upper_bound = 1000000):
    '''Returns a pd.DataFrame of fundamental data filtered by user input range
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - dataframe to be processed, contains tickers
    fundamental_indicator_key : str() - keyword for fundamental indicator requested
    
        Fundamental indicator keys ->
        
        P/E Ratio : [pe_ratio]
        EPS (Earnings per Share) : [eps]
        Annual Dividend : [dividend]
        Beta (vs. S&P 500) : [beta]
        EBIDT : [ebidt]
        Quick Ratio : [quick_ratio]
        3 Year Revenue Growth : [rev_growth]
        Free Cash Flow : [cash_flow]
    
    lower_bound : int() or float() - lower bound for fundamental value filter, default = -1000000
    upper_bound : int() or float() - upper bound for fundamental value filter, default = 1000000
     
     
     
    Returns
    -------
    result_df : pd.DataFrame - dataframe with ticker and filtered fundamental data
    '''

    fund_indicators_dict = {
        'pe_ratio' : 'peNormalizedAnnual',
        'eps' : 'epsNormalizedAnnual',
        'dividend' : 'dividendsPerShareTTM',
        'beta' : 'beta',
        'ebidt' : 'ebitdPerShareTTM',
        'quick_ratio' : 'quickRatioAnnual',
        'rev_growth' : 'revenueGrowth3Y',
        'free_cash_flow' : 'freeCashFlowAnnual'   
    }
    
    # Declate tickers list
    tickers_list = tickers_df['Symbol']
    
    # Declare fundamental data df and filter by ticker df
    data_df = stock_fundamentals_df[stock_fundamentals_df.symbol.isin(tickers_list)]
    data_df = data_df.set_index(['symbol'])
    
    # Extract requested fundamental data
    result_df = data_df[data_df['metric_type'] == fund_indicators_dict[fundamental_indicator_key]]
    
    # Convert all df values to numeric
    result_df['metric'] = result_df['metric'].apply(pd.to_numeric)
    
    # Filter fundamental data by user input range (lower_bound and upper_bound)
    result_df = result_df[(result_df['metric'] > lower_bound) & (result_df['metric'] < upper_bound)]
    
    # Clean up result_df
    result_df = result_df.drop(columns = ['metric_type', 'series'])
    result_df = result_df.rename(columns = {'symbol' : 'ticker', 
                                'metric' : fundamental_indicator_key})
    
    
    return result_df


# Method test
print(fundamental_data_query(sp500_tickers, 'eps', 0, 1))
# help(fundamental_data_query)

            eps
symbol         
AES     0.92046
AMCR    0.44298
BSX     0.42214
BMY     0.30596
COG     0.50169
CAH     0.65853
XRAY    0.31524
DLR     0.65652
DRE     0.63177
ENPH    0.94417
F       0.40347
FCX     0.37090
GILD    0.09739
PEAK    0.00892
HPE     0.76484
HWM     0.75740
IR      0.83537
KMI     0.60177
L       0.00624
LUMN    0.73884
MAR     0.56308
TAP     0.96720
NWL     0.55081
NLSN    0.89208
NI      0.91716
NLOK    0.93733
NXPI    0.34125
PRGO    0.59482
PSX     0.36630
PRU     0.07441
REG     0.88472
ROL     0.53295
ROST    0.78439
SBAC    0.64863
NOW     0.76813
TSLA    0.63712
TJX     0.50911
VTRS    0.71345
WELL    0.02194


### Traditional

In [ ]:
# Please start with calculating the Sharpe Ratio Calculation
# your method must take entire dataframe and return a dataframe with ticker and sharpe ratio for the given time
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)

# Method test

### Portfolio Methods

In [ ]:
# Portfolio weighing
def fundamental_portfolio_builder(tickers_df, start_date, end_date):
     '''Returns a pd.DataFrame time series price data of the equal weight portfolio
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - dataframe to be processed
    start_date : str() - string with date in following format YYYY-MM-DD
    end_date: str() - string with date in following format YYYY-MM-DD 
     
     
     
    Returns
    -------
    result_df : pd.DataFrame - dataframe with ticker and filtered fundamental data
    '''
    

## Visualization

In [ ]:
# your code here

## Dashboard

In [ ]:
# your code here